<h1>BUSINESS PROBLEM</h1>

> The WHO has stated that approximately 1.35 million people die each year as a result of road traffic crashes and cost most countries 3% of their gross domestic product, most of these are caused by cars.Sparks a BIG CONCERN RIGHT? There can be many reasons for car accidents, ranging from our negligence, weather conditions, the timing and many more.But wont it be great if we already know what can cause an accident? and how severe it could be so that we can avoid them at all costs. These accidents not only cause property damage but sometimes injuries and fatalities too.
The motive of my project is to find out the reasons and check how severe they can be.  

<h1> DATA </h1>

>The Accident dataset chosen has a lot of attributes that tell us about the severity of these accidents like Road_type,Speed_limit,Weather_conditions,Light_conditions,Day_of_Week,Special_Conditions_at_Site and a few others that will help us decide the severity of the accidents. Our predictor variable will be 'Accident_Severity' because it is used measure the severity of an accident from 1 to 3 within the dataset.
<ul>
<li>1:SLIGHT
<li>2:SERIOUS
<li>3:FATAL
</ul>
these codes will decide how severe the accident could be given the conditions

<H1>METHODOLOGY</H1>
UK Road Safety data: Total accident counts with accident severity as Slight, Serious and Fatal Normalized accident counts each month for slight and (Serious and Fatal clubbed) Plotting importance of each feature for considered features

Data Pre-processing techniques: The dataset is imputed by replacing NaN and missing values with most frequent values of the corresponding column. All the categorical values have been labeled by integers from 0 to n for each column. Time has been converted to categorial feature with 2 values i.e., daytime and night time.

The data is visualized for correlation. Negatively correlated features are selected to be dropped. Feature importance is plotted to visualize and only features with high importance are taken into consideration for predicting accident severity. The multi class label is converted to binary class by merging “Serious” and “Fatal” to Serious class.

Feature Selection: The dataset has 34 attributes describing the incident of an accident. There are mixed types of data such as continuous and categorical. Manually dropped few columns due to its inconsistency in values such as Accident ID, and Location ID. For selecting the best features, below functions are used from sklearn library.
<br>1.SelectKBest: SelectKBest is a sci-kit learn library provides the k best features by performing statistical tests i.e., chi squared computation between two non-negative features. Using chi squared function filters out the features which are independent of target attribute.
<br>2.Recursive Feature Elimination (RFE): RFE runs the defined model by trying out different possible combinations of features, and it removes the features recursively which are not impacting the class label. Logistic regression algorithm is used as a parameter for RFE to decide on features.

In [2]:

import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix  


In [9]:
df = pd.read_excel("/Users/charvimadan/Desktop/python course/Accident_Information.xls")

In [12]:
encoding = {
"Carriageway_Hazards": {"None": 0, "Other object on road": 1, "Any animal in carriageway (except ridden horse)": 1,  "Pedestrian in carriageway - not injured": 1, "Previous accident": 1, "Vehicle load on road": 1,  "Data missing or out of range": 0  }
}
df.replace(encoding, inplace=True)
print(df['Carriageway_Hazards'].value_counts())

0    64609
1      926
Name: Carriageway_Hazards, dtype: int64


In [13]:
print(df['Light_Conditions'].value_counts())
encoding_light = {"Light_Conditions": {"Daylight": 0, "Darkness - lights lit": 1, "Darkness - no lighting": 1, "Darkness - lighting unknown": 1, "Darkness - lights unlit": 1, "Data missing or out of range": 0}}
df.replace(encoding_light, inplace=True)
print(df['Light_Conditions'].value_counts())

Daylight                       47489
Darkness - lights lit          15895
Darkness - no lighting          1613
Darkness - lighting unknown      281
Darkness - lights unlit          257
Name: Light_Conditions, dtype: int64
0    47489
1    18046
Name: Light_Conditions, dtype: int64


In [14]:

print(df['Day_of_Week'].value_counts())
encoding_day_of_week = {"Day_of_Week": {"Saturday": 1, "Sunday": 1, "Monday": 0, "Tuesday": 0, "Wednesday": 0, "Thursday": 0, "Friday": 0}}
df.replace(encoding_day_of_week, inplace=True)
print(df['Day_of_Week'].value_counts())

Friday       10672
Wednesday    10008
Tuesday       9845
Thursday      9724
Monday        9156
Saturday      8896
Sunday        7234
Name: Day_of_Week, dtype: int64
0    49405
1    16130
Name: Day_of_Week, dtype: int64


In [15]:

print(df['Special_Conditions_at_Site'].value_counts())
encoding_Special_Conditions_at_Site = {"Special_Conditions_at_Site": {"None": 0, "Roadworks": 1, "Oil or diesel": 1, "Mud": 1, "Road surface defective": 1, "Auto traffic signal - out": 1, "Road sign or marking defective or obscured": 1, "Auto signal part defective": 1, "Data missing or out of range": 0}}
df.replace(encoding_Special_Conditions_at_Site, inplace=True)
print(df['Special_Conditions_at_Site'].value_counts())

None                                          64133
Roadworks                                       695
Oil or diesel                                   215
Road surface defective                          121
Road sign or marking defective or obscured      117
Auto traffic signal - out                       112
Mud                                              97
Auto signal part defective                       37
Data missing or out of range                      8
Name: Special_Conditions_at_Site, dtype: int64
0    64141
1     1394
Name: Special_Conditions_at_Site, dtype: int64


In [16]:
encoding_1st_road_class = {"1st_Road_Class": {"A": 1, "A(M)": 1, "B": 2, "C": 3, "Motorway": 4, "Unclassified": 1}}
df.replace(encoding_1st_road_class, inplace=True)
df['1st_Road_Class'].value_counts()

1    50153
2     6741
3     6735
4     1906
Name: 1st_Road_Class, dtype: int64

In [17]:

#replacing 'Data missing or out of range' with most occured value 'Give way or uncontrolled'
df['Junction_Control'] = df['Junction_Control'].replace(['Data missing or out of range'], 'Give way or uncontrolled')

In [18]:
df['Junction_Control'].value_counts()

Give way or uncontrolled               55043
Auto traffic signal                     9801
Stop sign                                431
Authorised person                        133
Not at junction or within 20 metres      127
Name: Junction_Control, dtype: int64

In [19]:

encoding_junction_detail = {"Junction_Control": 
                            {"Give way or uncontrolled": 1,
                             "Auto traffic signal": 2,
                             "Not at junction or within 20 metres": 3, 
                             "Stop sign": 4,
                             "Authorised person": 5,
                              }}
df.replace(encoding_junction_detail, inplace=True)
df['Junction_Control'].value_counts()

1    55043
2     9801
4      431
5      133
3      127
Name: Junction_Control, dtype: int64

In [20]:
encoding_junction_detail = {"Junction_Detail": 
                            {"Not at junction or within 20 metres": 1,
                             "T or staggered junction": 2,
                             "Crossroads": 3, 
                             "Roundabout": 4,
                             "Private drive or entrance": 5,
                             "Other junction": 6,
                             "Slip road": 7,
                             "More than 4 arms (not roundabout)": 8,
                             "Mini-roundabout": 9,
                             "Data missing or out of range": 1 }}
df.replace(encoding_junction_detail, inplace=True)
df['Junction_Detail'].value_counts()

2    23766
1    22063
3     8856
4     4203
6     2535
5     1462
8     1171
7      827
9      652
Name: Junction_Detail, dtype: int64

In [21]:
encoding_road_surface_cond = {"Road_Surface_Conditions": 
                            {"Dry": 1,
                             "Wet or damp": 2,
                             "Frost or ice": 3, 
                             "Snow": 4,
                             "Flood over 3cm. deep": 5,
                             "Data missing or out of range": 1 }}
df.replace(encoding_road_surface_cond, inplace=True)
df['Road_Surface_Conditions'].value_counts()

1    47654
2    16871
3      672
4      284
5       54
Name: Road_Surface_Conditions, dtype: int64

In [22]:
encoding_road_type = {"Road_Type": 
                            {"Single carriageway": 1,
                             "Dual carriageway": 2,
                             "Roundabout": 3, 
                             "One way street": 4,
                             "Slip road": 5,
                             "Unknown": 0,
                             "Data missing or out of range": 1 }}
df.replace(encoding_road_type, inplace=True)
df['Road_Type'].value_counts()

1    50127
2     9428
3     3236
4     1716
5      520
0      508
Name: Road_Type, dtype: int64

In [23]:
encoding_urban_rural = {"Urban_or_Rural_Area": 
                            {"Urban": 1,
                             "Rural": 2,
                             "Unallocated": 1 }}
df.replace(encoding_urban_rural, inplace=True)
df['Urban_or_Rural_Area'].value_counts()

1    52589
2    12946
Name: Urban_or_Rural_Area, dtype: int64

In [24]:
encoding_weather = {"Weather_Conditions": 
                            {"Fine no high winds": 1,
                             "Raining no high winds": 2,
                             "Raining + high winds": 3,
                             "Fine + high winds": 4,
                             "Snowing no high winds": 5,
                             "Fog or mist": 6,
                             "Snowing + high winds": 7,
                             "Unknown": 1,
                             "Other": 1,
                             "Data missing or out of range": 1 }}
df.replace(encoding_weather, inplace=True)
df['Weather_Conditions'].value_counts()

1    55972
2     7570
4      658
3      578
5      458
6      243
7       56
Name: Weather_Conditions, dtype: int64

In [25]:
np.where(np.isnan(df['Speed_limit']))

(array([], dtype=int64),)

In [26]:
df['Speed_limit'].fillna((df['Speed_limit'].mean()), inplace=True)

In [27]:
df['Time'].fillna(0, inplace=True)

In [28]:
def period(row):
    rdf = []
    if(type(row) == float):
        row = str(row)
        rdf = row.split(".")
    else:
        rdf = str(row).split(":"); # day -- 8am-8pm
        
    hr = rdf[0]
    if int(hr) > 8 and int(hr) < 20:
        return 1;
    else:
        return 2;

In [29]:
df['Time'] = df['Time'].apply(period)

In [30]:
df_train1 = df[['1st_Road_Class','Carriageway_Hazards','Junction_Control','Day_of_Week','Junction_Detail','Light_Conditions','Road_Surface_Conditions','Road_Type','Special_Conditions_at_Site','Speed_limit','Time','Urban_or_Rural_Area','Weather_Conditions','Accident_Severity']]

In [31]:
df_slight = df_train1[df_train1['Accident_Severity']=='Slight']

In [32]:
df_serious = df_train1[df_train1['Accident_Severity']=='Serious']

In [33]:
df_fatal = df_train1[df_train1['Accident_Severity']=='Fatal']

In [34]:

df_serious['Accident_Severity'].value_counts()

Serious    8197
Name: Accident_Severity, dtype: int64

In [35]:

random_subset = df_slight.sample(n=3)
random_subset.head()

,1st_Road_Class,Carriageway_Hazards,Junction_Control,Day_of_Week,Junction_Detail,Light_Conditions,Road_Surface_Conditions,Road_Type,Special_Conditions_at_Site,Speed_limit,Time,Urban_or_Rural_Area,Weather_Conditions,Accident_Severity
5287,1,0,1,0,1,0,1,1,0,30,1,1,1,Slight
37794,3,0,1,0,1,1,2,1,0,30,1,1,1,Slight
4561,1,0,2,0,3,1,1,1,0,30,1,1,1,Slight


In [36]:
df_fatal['Accident_Severity'].value_counts()

Fatal    740
Name: Accident_Severity, dtype: int64

In [45]:
df_slight_sampling = df_slight.sample(n=8937)  
#Matched the combined number of records for Fatal and Serious(As we are going to club fatal&serious to Serious)

In [46]:

df_serious_sampling = df_serious.sample(n=740)  #Matched number of records with the rarer class 

In [47]:
df_final_sampling = pd.concat([df_serious_sampling,df_slight_sampling,df_fatal])

In [48]:
df_final_sampling.head()

,1st_Road_Class,Carriageway_Hazards,Junction_Control,Day_of_Week,Junction_Detail,Light_Conditions,Road_Surface_Conditions,Road_Type,Special_Conditions_at_Site,Speed_limit,Time,Urban_or_Rural_Area,Weather_Conditions,Accident_Severity
24990,2,0,1,0,1,0,1,1,0,30,2,1,1,Serious
19161,1,0,1,0,1,0,1,1,0,30,2,1,1,Serious
31570,1,0,1,1,2,0,2,1,0,30,1,1,2,Serious
47236,3,1,1,1,6,0,1,0,1,30,1,2,1,Serious
30517,2,0,1,1,2,0,1,1,0,40,1,2,1,Serious


In [49]:
df_test = df_final_sampling[['Accident_Severity']]

In [51]:
df_train = df_final_sampling[['1st_Road_Class','Carriageway_Hazards','Junction_Control','Day_of_Week','Junction_Detail','Light_Conditions','Road_Surface_Conditions','Road_Type','Special_Conditions_at_Site',
                              'Speed_limit','Time','Urban_or_Rural_Area','Weather_Conditions']]

In [52]:
df_test['Accident_Severity'].value_counts()

Slight     8937
Fatal       740
Serious     740
Name: Accident_Severity, dtype: int64

<h2>results</h2>

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train, df_test, test_size=0.2)

In [54]:
from sklearn.ensemble import RandomForestClassifier
#class_weight = dict({2:1, 1:15, 0:50})
rdf = RandomForestClassifier(n_estimators=300,random_state=35)

rdf.fit(X_train,y_train)

y_pred=rdf.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


Accuracy: 0.8430902111324377
[[  17    3  126]
 [   9    2  143]
 [  34   12 1738]]
              precision    recall  f1-score   support

       Fatal       0.28      0.12      0.17       146
     Serious       0.12      0.01      0.02       154
      Slight       0.87      0.97      0.92      1784

   micro avg       0.84      0.84      0.84      2084
   macro avg       0.42      0.37      0.37      2084
weighted avg       0.77      0.84      0.80      2084



In [55]:

from sklearn.ensemble import RandomForestClassifier
#class_weight = dict({2:1, 1:15, 0:50})
rdf = RandomForestClassifier(bootstrap=True,
            class_weight="balanced_subsample", 
            criterion='gini',
            max_depth=8, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=300,
            oob_score=True,
            random_state=35,
            verbose=0, warm_start=False)

In [56]:
rdf.fit(X_train,y_train)

y_pred=rdf.predict(X_test)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


In [57]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.5930902111324377


In [58]:
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))

[[  73   24   49]
 [  52   29   73]
 [ 343  307 1134]]
              precision    recall  f1-score   support

       Fatal       0.16      0.50      0.24       146
     Serious       0.08      0.19      0.11       154
      Slight       0.90      0.64      0.75      1784

   micro avg       0.59      0.59      0.59      2084
   macro avg       0.38      0.44      0.37      2084
weighted avg       0.79      0.59      0.66      2084



In [61]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [62]:
from xgboost import XGBClassifier
model = XGBClassifier(learning_rate =0.07, n_estimators=300,
                      class_weight="balanced_subsample",
                      max_depth=8, min_child_weight=1,
                      scale_pos_weight=7,
                      seed=27,subsample=0.8,colsample_bytree=0.8)

model.fit(X_train,y_train)

y_pred=model.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

XGBoostError: XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed (vcomp140.dll or libgomp-1.dll for Windows, libomp.dylib for Mac OSX, libgomp.so for Linux and other UNIX-like OSes). Mac OSX users: Run `brew install libomp` to install OpenMP runtime.
  * You are running 32-bit Python on a 64-bit OS
Error message(s): ['dlopen(/anaconda3/lib/python3.7/site-packages/xgboost/lib/libxgboost.dylib, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib\n  Referenced from: /anaconda3/lib/python3.7/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: image not found']


In [63]:
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))

[[  73   24   49]
 [  52   29   73]
 [ 343  307 1134]]
              precision    recall  f1-score   support

       Fatal       0.16      0.50      0.24       146
     Serious       0.08      0.19      0.11       154
      Slight       0.90      0.64      0.75      1784

   micro avg       0.59      0.59      0.59      2084
   macro avg       0.38      0.44      0.37      2084
weighted avg       0.79      0.59      0.66      2084



In [64]:
# import the class
from sklearn.neighbors import KNeighborsClassifier

# instantiate the model (with the default parameters)
knn = KNeighborsClassifier(n_neighbors=3,weights='distance')

# fit the model with data (occurs in-place)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


[[  24    6  116]
 [  17    8  129]
 [ 109   42 1633]]
              precision    recall  f1-score   support

       Fatal       0.16      0.16      0.16       146
     Serious       0.14      0.05      0.08       154
      Slight       0.87      0.92      0.89      1784

   micro avg       0.80      0.80      0.80      2084
   macro avg       0.39      0.38      0.38      2084
weighted avg       0.77      0.80      0.78      2084



In [66]:

from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
y_pred = logisticRegr.predict(X_test)
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))

[[   0    0  146]
 [   0    0  154]
 [   0    0 1784]]
              precision    recall  f1-score   support

       Fatal       0.00      0.00      0.00       146
     Serious       0.00      0.00      0.00       154
      Slight       0.86      1.00      0.92      1784

   micro avg       0.86      0.86      0.86      2084
   macro avg       0.29      0.33      0.31      2084
weighted avg       0.73      0.86      0.79      2084



/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [67]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print(accuracy_score(y_test, y_pred))
confusion_matrix(y_test, y_pred)
print(format(classification_report(y_test, y_pred)))

0.7855086372360844
              precision    recall  f1-score   support

       Fatal       0.16      0.29      0.21       146
     Serious       0.00      0.00      0.00       154
      Slight       0.87      0.89      0.88      1784

   micro avg       0.79      0.79      0.79      2084
   macro avg       0.35      0.39      0.36      2084
weighted avg       0.76      0.79      0.77      2084



/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [68]:

from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(loss="deviance", learning_rate=0.1, 
      n_estimators=100, subsample=1.0, criterion="friedman_mse", 
      min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
      max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, init=None, 
      random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, 
      presort="auto")

y_pred = gbc.fit(X_train, y_train.values.ravel()).predict(X_test)
print(format(classification_report(y_test, y_pred)))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

       Fatal       0.00      0.00      0.00       146
     Serious       1.00      0.01      0.01       154
      Slight       0.86      1.00      0.92      1784

   micro avg       0.85      0.85      0.85      2084
   macro avg       0.62      0.33      0.31      2084
weighted avg       0.81      0.85      0.79      2084

0.8546065259117083


<h2>DISCUSSION</h2>


Our main aim was to predict the severity of the accident when it is “serious” and “fatal”. It was very difficult to handle this large-sized data. Using HPC we were able to run most of our algorithms. Data is highly imbalanced so even though most of our algorithms were giving > 89% accuracies, it was of no use. It was predicting all the accidents as slight accidents. After checking on all these algorithms, the team even tried dimensionality reduction techniques and but the results were not improved. Then the team decided to use the undersampled dataset as it was giving better results in predicting the severe/fatal accidents. This decision was made on trying out oversampling, undersampling, test and train data with an equal ratio of classification classes

<h2>Conclusion</h2>


In conclusion, most of the algorithms are biased towards most frequent class. However, efficient pre-processing and corresponding imbalanced data techniques should give optimal results. Based on the current known conditions of weather, light, traffic signal, road surface, speed limit etc., accident severity can be classified. But there is no one feature, that influences the accident severity.